# logging Usage

## 日志基础教程

日志是对软件执行时所发生事件的一种追踪方式。软件开发人员对他们的代码添加日志调用，借此来指示某事件的发生。一个事件通过一些包含变量数据的描述信息来描述。开发者还会区分事件的重要性，重要性也被称为等级或严重性。

|你想要执行的任务|此任务最好的工具|
|---------------|--------------|
|结果显示在控制台|print()|
|提交事件报告|logging.info()，当有诊断目的，需要详细输出信息时使用logging.debug()。
|提出一个警告信息基于一个特殊的运行时事件|warnings.warn()需要修改应用以消除警告，logging.warning()需要引起关注|
|报告错误|引发异常，raise Error|
|报告错误而不引发异常|logging.error(),logging.exception()或logging.critical()|

日志功能应根据事件的级别或严重性选定。各级别适用性如下（以严重性递增）：
|级别|何时使用|
|----|-------|
|DEBUG|细节信息，仅当诊断问题时适用。|
|INFO|确认程序按预期运行。|
|WARNING|表明有已经或即将发生的意外（如：磁盘空间不足）。程序仍按预期进行。|
|ERROR|由于严重的问题，程序的某些功能已经不能正常执行|
|CRITICAL|严重的错误，程序已不能继续执行|

默认的级别是WARNING，意味着只会追踪该级别及以上的事件，除非更改日志配置。

所追踪事件可以以不同形式处理。输出控制台或写入磁盘文件。

In [1]:
import logging

一个简单的例子，因为默认的级别是WARNING，所以不会显示INFO的消息。

In [3]:
logging.warning('Watch out!')  # will print a message to the console
logging.info('I told you so')  # will not print anything

### 记录日志到文件
第一次记录日志信息后自动配置日志信息，且无法更改配置。如需更改，需要重新导入logging模块。

In [20]:
# import reload
from importlib import reload
reload(logging)

<module 'logging' from 'c:\\Users\\95194\\miniconda3\\lib\\logging\\__init__.py'>

In [7]:
logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.DEBUG)
logging.debug('This message should go to the log file')
logging.info('So should this')
logging.warning('And this, too')
logging.error('And non-ASCII stuff, too, like Øresund and Malmö')

### 从命令行设定日志级别

In [8]:
# 假设从命令行得到了loglevel参数，可以通过下面的方式来设置logging的level
loglevel = args.loglevel
numeric_level = getattr(logging, loglevel.upper(), None)
if not isinstance(numeric_level, int):
    raise ValueError('Invalid log level: %s' % loglevel)
logging.basicConfig(..., level=numeric_level)

NameError: name 'loglevel' is not defined

通过指定filemode控制文件写入方式是追加（a）还是替换（w)。

In [ ]:
logging.basicConfig(filename='example.log', filemode='w', level=logging.DEBUG)

### 从多个模块记录日志
如果程序包含多个模块，这是一个如何组织日志记录的示例：

In [9]:
# myapp.py
import logging
import mylib

def main():
    logging.basicConfig(filename='myapp.log', level=logging.INFO)
    logging.info('Started')
    mylib.do_something()
    logging.info('Finished')

if __name__ == '__main__':
    main()

In [ ]:
# mylib.py
import logging

def do_something():
    logging.info('Doing something')

### 记录变量数据
要记录变量数据，请使用格式字符串作为事件描述消息，并附加传入变量数据作为参数。例如：

In [11]:
logging.warning('%s before you %s', 'Look', 'leap!')

适用%-s形式的字符串格式化方法，是为了向后兼容。也可以适用较新的格式化选项例如:`str.format()`和`string.Template`。

### 更改显示消息的格式

In [18]:
logging.basicConfig(format='%(levelname)s %(asctime)s %(message)s', level=logging.DEBUG)
logging.debug('This message should appear on the console')
logging.info('So should this')
logging.warning('And this, too')

DEBUG 2023-02-05 18:09:58,511 This message should appear on the console
INFO 2023-02-05 18:09:58,512 So should this
WARNING 2023-02-05 18:09:58,514 And this, too


可以通过`datefmt`控制日期时间的格式，格式与`time.strftime()`相同

In [21]:
logging.basicConfig(format='%(levelname)s %(asctime)s %(message)s', level=logging.DEBUG, datefmt='%m/%d/%Y %I:%M:%S %p')
logging.debug('This message should appear on the console')

DEBUG 02/05/2023 06:12:34 PM This message should appear on the console


## 进阶日志教程
日志库采用模块化方法，并提供几类组件：记录器、处理器、过滤器和格式器。
- 记录器暴露了应用程序代码直接使用的接口。
- 处理器将日志记录（由记录器创建）发送到适当的目标。
- 过滤器提供了更细粒度的功能，用于确定要输出的日志记录。
- 格式器指定最终输出中日志记录的样式。